# GTEx V8 genotypes
Save V8 genotype data to HDF5 for association analysis.

Genotype here already have been imputed and filtered for MAF > 1%. I only need to convert VCF to HDF5 format for per-gene data. 

In [1]:
[global]
cwd = "~/Documents/GTExV8"

## Convert data to PLINK format

In [9]:
%sosrun vcf_to_plink -b ~/Documents/GTExV8/utils
[vcf_to_plink_1]
# VCF to plink rough conversion
depends: executable("plink")
input: "${cwd!a}/GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze_MAF001_GTonly.vcf.gz"
output: "${input!nn}.a2flipped.bed"
task: workdir = cwd
run:
  plink --vcf ${input} --out ${output!n} --make-bed

[vcf_to_plink_2]
# Fix multi-allelic sites
# By changing duplicate SNP ID
output: "${input!n}.bim"
task: workdir = cwd
python:
    from collections import Counter
    import numpy as np
    dat = np.genfromtxt(${output!r}, dtype = '<U48')
    counter = Counter()
    dup_list = dat[:,1]
    deduped = []
    for name in dup_list:
        new = name + '_' + str(counter[name]) if counter[name] else name
        counter.update({name: 1})
        deduped.append(new)
    dat[:,1] = np.array(deduped)
    np.savetxt(${output!r}, dat, delimiter = '\t', fmt = '%s')

[vcf_to_plink_3]
# Allele fix
input: group_by = 'single'
output: "${input!nn}.bed"
task: workdir = cwd
bash:
  paste <(cut -f2 ${input!n}.bim) <(zcat ${input!nn}.vcf.gz | grep -v "^#" | cut -f4) > ${input!nn}.a2
  plink --bfile ${input!n} \
        --a2-allele ${input!nn}.a2 2 1 '#' \
        --out ${output!n} --make-bed
  rm -f ${input!nn}.a2

INFO: vcf_to_plink_1 (index=0) is ignored due to saved signature


1 task completed: ca3b

"<i id=""status_localhost_9e76872946e33a65442046fe2b28c705"" class=""fa fa-2x fa-fw fa-square-o"" onmouseover=""$('#status_localhost_9e76872946e33a65442046fe2b28c705').addClass('fa-stop').removeClass('fa-square-o')"" onmouseleave=""$('#status_localhost_9e76872946e33a65442046fe2b28c705').addClass('fa-square-o').removeClass('fa-stop')"" onclick=""kill_task('9e76872946e33a65442046fe2b28c705', 'localhost')"" >",9e76872946e33a65442046fe2b28c705,,0 sec


## Create cisSNP genotype data files
This is largely copied from `20170530_CisSNP.ipynb`

### Annotate genes
Code chunk below annotates genes to chromosomal positions. Genes are taken from the GTEx expression file.

In [4]:
%sosrun ensembl_annotation
[ensembl_annotation]
# Input is GTEx expression file with first column the ENSG IDs
depends: R_library("biomaRt")
input: "${cwd!a}/rna-seq/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_reads.gct.gz"
output: "${cwd!a}/${input!bn}.annotation"
task: workdir = cwd
bash:
    zcat ${input} | cut -f 1 | tail -n+4 | cut -f 1 -d '.' > ${input!n}.gene_id

R:
    values = unlist(read.table("${input!n}.gene_id", stringsAsFactors=FALSE)) 
    ensembl = biomaRt::useMart("ensembl", dataset="hsapiens_gene_ensembl", host="www.ensembl.org")
    res = biomaRt::getBM(attributes = c("chromosome_name", "start_position", "end_position", "ensembl_gene_id", "hgnc_symbol"), filters = c("ensembl_gene_id"), values = values,  mart = ensembl)
    write.table(res, file = ${output!r}, row.names=FALSE, na= "<NA>", col.names = FALSE)

bash:
    cat ${output} | sed -e 's/"//g' | sort -g -o ${output}
    rm -f ${input!n}.gene_id

1 task completed: 316b

### Make cis-SNP tables
Using position annotations previously obtained, we work though the genotype data in PLINK format, , creating for each gene a table containing its cis-SNPs. Variants are first annotated to genes, then cis-SNPs to extract are defined as SNPs 1,000,000 bp up/downstream of the gene's TSS site. The outcome is a **single analysis ready file** in HDF5 format containing ~50K groups of genotype data (gene-names). Code chunk below implements the procedure.

In [ ]:
%sosrun cis_groups_hdf5
[cis_groups_hdf5]
# Convert plink genotype to HDF5 in batches of genes
# Only autosomal genes are considered
# cis: ${cis} up/downstream from TSS of each gene
depends: Py_Module("pandas-plink"), Py_Module("dask"), 
    "${cwd!a}/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_reads.gct.annotation"
parameter: cis = 1000000
parameter: resume = 1
input: "${cwd!a}/GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze_MAF001_GTonly.bed"
output: "${cwd!a}/GTExV8.genotype.cis.h5"
task: workdir = cwd
python:
    import os
    import pandas as pd, numpy as np
    from pandas_plink import read_plink
    keys = []
    if os.path.isfile(${output!r}):
       if ${resume} == 1:
          print("loading existing database ${output!b}")
          keys = pd.HDFStore(${output!r}).keys()
          print("{} existing genes will be skipped".format(len(keys)))
       else:
          os.remove(${output!r})

    (bim, fam, bed) = read_plink(${input!nr})
    ann = [x.strip().split() for x in open("${cwd!a}/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_reads.gct.annotation").readlines()]
    n = len(ann)
    empty_genes = []
    for i, line in enumerate(ann):
        if(i % 100 == 0):
            print('[percent completed] %.2f' % ((float(i)/n)*100))
        if line[0] not in list(map(str, [x+1 for x in range(22)])):
            continue
        if '/chr{}/{}'.format(line[0], line[3]) in keys:
            continue
        snps = bim.query("chrom == '{}' and (pos >= {} and pos <= {})".\
                                format(line[0], int(line[1]) - ${cis}, int(line[1]) + ${cis}))
        if (snps.empty):
            empty_genes.append(line[3])
            continue
        X = pd.DataFrame(2 - bed[snps.i,:].compute(), 
                         index = [':'.join((x.split('_', 1)[0], y,z)) for x, y, z in zip(snps.snp, snps.a1, snps.a0)], 
                         columns = fam.iid, dtype = np.uint8)
        X.to_hdf(${output!r}, 'chr{}/{}'.format(line[0], line[3]), mode = 'a', complevel = 9, complib = 'zlib')
    with open("${output!n}.log", 'w') as f:
         f.write('# Empty genes\n' + '\n'.join(empty_genes))